In [1]:
import re
import nltk
import string
import heapq
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BASHA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BASHA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Processing Text

In [2]:
original_text = '''artificial intelligence is human like intelligence.
                   It is the study of intelligent artificial agents.
                   Science and engineering to produce intelligent machines.
                   Solve problems and have intelligence.
                   Related to intelligent behavior.
                   Developing of reasoning machines.
                   Learn from mistakes and successes.
                   Artificial intelligence is related to reasoning in everyday situations.'''

original_text =re.sub(r'\s+',' ',original_text) # \s fro space and + for one or more than one space

original_text

'artificial intelligence is human like intelligence. It is the study of intelligent artificial agents. Science and engineering to produce intelligent machines. Solve problems and have intelligence. Related to intelligent behavior. Developing of reasoning machines. Learn from mistakes and successes. Artificial intelligence is related to reasoning in everyday situations.'

In [3]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [4]:
def preprocess(text):
    formatted_text = text.lower()
    tokens = []  # storing each words of sentence with out stop words
    for token in nltk.word_tokenize(formatted_text):
        tokens.append(token)
    
    tokens = [word for word in tokens if word not in stopwords and word not in string.punctuation]
    formatted_text = ' '.join(element for element in tokens) # creating sentence without stop words and punctations
    
    return formatted_text

In [5]:
formatted_text = preprocess(original_text)
print(formatted_text)

artificial intelligence human like intelligence study intelligent artificial agents science engineering produce intelligent machines solve problems intelligence related intelligent behavior developing reasoning machines learn mistakes successes artificial intelligence related reasoning everyday situations


In [22]:
def calualte_sentence_score(sentences, important_words,distance):
    scores = []
    sentence_index = 0
    for sentence in [nltk.word_tokenize(sentence) for sentence in sentences]:
        word_index=[]
        for word in important_words:
            try:
                word_index.append(sentence.index(word))
            except ValueError:
                pass
        
        word_index.sort()
        
        if len(word_index) == 0:
            continue
        
        groups_list = []
        group = [word_index[0]]
        
        i = 1
        
        while i < len(word_index):
            if word_index[i] - word_index[i-1] < distance:
                group.append(word_index[1])
            else:
                groups_list.append(group[:])
                group = [word_index[i]]
                
            i +=1
        groups_list.append(group)
        
        max_group_score = 0
        
        for g in groups_list:
            
            important_words_in_group = len(g)
            total_words_in_group = g[-1] - g[0] + 1
            score = 1.0 * important_words_in_group ** 2 / total_words_in_group
            
            if score > max_group_score :               
                max_group_score = score
                
        scores.append((max_group_score, sentence_index))
        sentence_index += 1
    
    return scores
            
        
      

In [27]:
def Summarize(text, top_num_words, distance, number_of_sentences):
    original_sentences = [sentence for sentence in nltk.sent_tokenize(text)]
    
    fromatted_sentences = [preprocess(original_sentence) for original_sentence in original_sentences]
    
    words = [word for sentence in fromatted_sentences for word in nltk.word_tokenize(sentence)]
    
    frequency = nltk.FreqDist(words)
    
    top_n_words= [word[0] for word in frequency.most_common(top_num_words)]
        
    sentenced_score = calualte_sentence_score(fromatted_sentences,top_n_words,distance)
    
    best_sentences = heapq.nlargest(number_of_sentences, sentenced_score)
    
    best_sentences = [original_sentences[i] for (score, i) in best_sentences]
    summary = " ".join(best_sentences)
    
    return summary  
    

In [32]:
Summarize(original_text,5,2,9)

'Learn from mistakes and successes. Related to intelligent behavior. Science and engineering to produce intelligent machines. It is the study of intelligent artificial agents. artificial intelligence is human like intelligence. Developing of reasoning machines. Solve problems and have intelligence.'

In [30]:
from goose3 import Goose
g = Goose()
url =  'https://www.reuters.com/world/americas/haitians-scramble-rescue-survivors-ruins-major-quake-2021-08-15/'
article = g.extract(url)

In [33]:
Summarize(article.cleaned_text,5,2,9)

'Haiti\'s Civil Protection Agency said the toll from the disaster had climbed to 1,297 and the hospitals that were still functioning were struggling to cope as some 5,700 people were injured. From the Vatican, Pope Francis urged the international community to show support swiftly. Colombia sent search and rescue personnel. "They are affected but resilient. Haiti\'s Prime Minister Ariel Henry, who flew to visit Les Cayes, praised the dignity shown by people there even in the midst of their suffering. Some 13,694 houses were destroyed, the civil protection agency said, suggesting the toll could rise further. Southwestern Haiti bore the brunt of the blow, especially in the region in and around the town of Les Cayes. PORT-AU-PRINCE, Aug 15 (Reuters) - Haiti\'s hospitals were swamped on Sunday by thousands of injured residents after a devastating earthquake the day before killed at least 1,297 people as authorities raced to bring doctors to the worst-hit areas before a major storm hits. Foo